This notebook cleans .txt and .dta data for students performance on exams for BIO1111 and exports a csv which can be analyzed in an R script.
#### Author: Christopher Agard

In [1]:
import pandas as pd
import numpy as np
import os,glob

In [51]:
def cleanSoarExam (data\
                   , examNum\
                   , fileType='flat'\
                   , colSpec = [(0,9),(10,21),(22,28),(29,30),(30,32),(32,34),(34,36),(36,38),(39,41),(41,68)]\
                  , soarSessions=[]
                  ,sheetname=0):
    import pandas as pd
    import numpy as np
    import os
    assert fileType in ['excel','flat'], "\n{} fileType is not currently supported.".format(fileType)
    numbers=pd.Series(list(range(28))).astype(str)
    itemNames= 'item_'+ numbers[1:]
    if fileType == 'flat':
        colnames = ['tuid','last','first','middle','testcode','unnamed2','unnamed3','soar','ncorrect','item']
        try:
            df=pd.read_fwf(data,colSpec,names=colnames)
            df['examNumber']=examNum
            itemData=df.item.apply(lambda i: pd.Series(list(i)))
            itemData.columns=itemNames
            df=df.merge(itemData,'outer',left_index=True,right_index=True).drop('item',axis=1)
            if soarSessions ==np.nan:
                df.loc['soarType']='other'
            else:
                df.loc[df.soar.isin(soarSessions),'soarType']='mine'
                df.loc[~df.soar.isin(soarSessions),'soarType']='other'
                df.loc[df.tuid=='NNNNNNNNN','soarType']='key'
        except:
            df = np.nan
            "print(\nCould not find file {} or {} was not acceptable value for colSpec)".format(data,colSpec)
    elif fileType=='excel':
        colnames = ['tuid','last','first','middle','testcode','unnamed2','unnamed3','soar','ncorrect']+[itemNames]
        try:
            df=pd.read_excel(data,sheet_name=sheetname, names=colnames)
            df['examNumber']=examNum
            if soarSessions ==np.nan:
                df.loc['soarType']='other'
            else:
                df.loc[df.soar.isin(soarSessions),'soarType']='mine'
                df.loc[~df.soar.isin(soarSessions),'soarType']='other'
                df.loc[df.tuid=='NNNNNNNNN','soarType']='key'
        except:
            df = np.nan
            "print(\nCould not find file {})".format(data)
    return df
        

numbers=pd.Series(list(range(28))).astype(str)
itemNames= 'item_'+ numbers[1:]
colnames = ['tuid','last','first','middle','testcode','unnamed2','unnamed3','soar','ncorrect'] +[itemNames]
colnames

In [55]:
colnames = ['tuid','last','first','middle','testcode','unnamed2','unnamed3','soar','ncorrect']+[itemNames]

In [54]:
cleanSoarExam(glob.glob('*.xlsx'[0]),fileType='excel',examNum=2,soarSessions=[73,74,75])

C:\Users\tuh27554\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


nan

In [63]:
myfile = glob.glob('*.xlsx')[0]
# print(myfile)
test = pd.read_excel(myfile,header = None,examNum=2,soarSessions=[73,74,75])
colnames = ['tuid','last','first','middle','testcode','unnamed2','unnamed3','soar','ncorrect']+[itemNames]
test.columns = colnames
test

ValueError: Length mismatch: Expected axis has 35 elements, new values have 10 elements

# Setting source and output folders

In [23]:
worksource = "C:/Users/tuh27554/Documents/BIOL1111/Spr 2019/Raw Data/Exam 2"
workoutput = "C:/Users/tuh27554/Documents/BIOL1111/Spr 2019/Results/Exam 2"

# Getting and Cleaning Exam Data

In [ ]:
os.chdir(worksource)

In [24]:
df1=cleanSoarExam('Exam 2 1pm.csv',examNum=2,soarSessions=[73,74,75])
# df1['version'] = 1
df2=cleanSoarExam('Exam 2 4pm.csv',examNum=2,soarSessions=[73,74,75])
# df2['version'] = 2

UnboundLocalError: local variable 'res' referenced before assignment

In [21]:
df = df1.append(df2)
df.head()

UnboundLocalError: local variable 'res' referenced before assignment

Now we change the dir to the outputfolder and save the cleaned, merged file.

In [ ]:
os.chdir(workoutput)
df.to_csv("Spr19Exam 1_cleaned.csv")

# Analyzing Exam Data

In [ ]:
df.head()

# Most Frequently Wrong Answers

Here we identify the columns containing information about responses.

In [ ]:
itemcols = df.columns[df.columns.str.contains('item')]
itemcols

In [ ]:
df.soarType.value_counts()

In [ ]:
def countwrong (df,col,ver,truthcol = 'soarType',truthind = 'key',compind = ['mine'],vercol = 'version'):
    truth = df.loc[(df[truthcol]==truthind)&(df[vercol] == ver),col]
    comp = df.loc[(df[truthcol].isin(compind))&(df[vercol] == ver),col]
    res = pd.DataFrame(data ={'item':[col],'number':[(comp != truth[0]).sum()]})
    return(res)


In [ ]:
def countwrong_allcol(df,ver,itemcols,sortfields = ['number','item'],
                      truthcol = 'soarType',truthind = 'key',compind = 'mine',vercol = 'version'):
    tmp = pd.DataFrame()
    for col in itemcols:
        tmp = tmp.append(countwrong(df= df, col = col, ver = ver))
    tmp = tmp.sort_values(sortfields,ascending=False).reset_index(drop = True)
    return tmp

In [ ]:
v1wrong = countwrong_allcol(df,1,itemcols = df.columns[df.columns.str.contains('item')])
v2wrong = countwrong_allcol(df,2,itemcols = df.columns[df.columns.str.contains('item')])

Now we apply this function to create a ranked list of items by order of the number of my students getting them wrong.

In [ ]:
wrongcount_v1 = pd.DataFrame()
for col in itemcols:
    wrongcount_v1 = wrongcount_v1.append(countwrong(df= df, col = col, ver = 1,compind=['mine','other']))
wrongcount_v1 = wrongcount_v1.sort_values(['number','item'],ascending=False).reset_index(drop = True)
wrongcount_v1

In [ ]:
wrongcount_v2 = pd.DataFrame()
for col in itemcols:
    wrongcount_v2 = wrongcount_v2.append(countwrong(df= df, col = col, ver = 2,compind=['other','mine']))
wrongcount_v2 = wrongcount_v2.sort_values(['number','item'],ascending=False).reset_index(drop = True)
wrongcount_v2

## Sec 73

## Sec 74

## Sec 75

# Printing the Keys

In [ ]:
v1keys = df.loc[(df.tuid=='NNNNNNNNN')&(df.version==1),itemcols]
v1keys

In [ ]:
v2keys = df.loc[(df.tuid=='NNNNNNNNN')&(df.version==2),itemcols]
v2keys

In [ ]:
# v1keys = v1keys.T.rename(columns={0:'version1'})
# v2keys = v2keys.T.rename(columns={0:'version2'})
pd.DataFrame(data = {'version1':v1keys,'version2':v2keys},index = itemcols)

In [ ]:
letters2num_dict = {1:'D',2:'C',3:'B',4:'A'}
exkeys.version